In [2]:
#Поскольку multiprocessig не работает в моем ноутбуке, я основное повествование моей работы буду вести здесь, а в 
#multiproc_chtuki.py буду создавать какие либо тяжелые датасеты. Конечно же буду оставлять сноски
#Часть кода там закомментировано, чтобы не мешало. Не подумайте, что оно ненужное.
#И так
#Все свое нашу с собой
import numpy as np
import pandas as pd
import pickle as p
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Я выбрал датасет с песнями, поскольку других не нашел
data_artists = pd.read_csv('artists-data.csv')
data_lyrics = pd.read_csv('lyrics-data.csv')

In [ ]:
#я немного преобразовал эти 2 файла в один. Это я делал в py файле - №1 в цикле main
#Пришлось оставить только английские песенки т.к. словарями векторов на испанские, портагальские и т.п. слова я не имею, обидно

In [10]:
data = pd.read_csv('data.csv')

In [303]:
data 

,Artist,SName,Lyric,Song,Popularity
0,10000 Maniacs,More Than This,I could feel at the time. There was no way of ...,110.0,0.3
1,10000 Maniacs,Because The Night,"Take me now, baby, here as I am. Hold me close...",110.0,0.3
2,10000 Maniacs,These Are Days,These are. These are days you'll remember. Nev...,110.0,0.3
3,10000 Maniacs,A Campfire Song,"A lie to say, ""O my mountain has coal veins an...",110.0,0.3
4,10000 Maniacs,Everyday Is Like Sunday,Trudging slowly over wet sand. Back to the ben...,110.0,0.3
...,...,...,...,...,...
109552,Sambô,This Love,I was so high I did not recognize. The fire bu...,71.0,0.8
109553,Sambô,Valerie,Well sometimes I go out by myself. And I look ...,71.0,0.8
109554,Sambô,Wake Me Up,Feeling my way through the darkness. Guided by...,71.0,0.8
109555,Seu Jorge,Don't,"Don't, don't, that's what you say. Each time t...",106.0,4.2


In [ ]:
#Теперь обработаем песню, приведя слова к начальным формам соответсвующих частей речи, удалив знаки припенанния и stopwords.
#Новый столбик - №2
#Еще один столбик - похожий на имя артиста

In [6]:
data = data.drop_duplicates().reset_index(drop=True)
data['Artist_clear'] = data['Artist'].apply(lambda a: a.lower())

In [11]:
data

,Artist,SName,Lyric,Song,Popularity,Lyric_clear,Artist_clear
0,10000 Maniacs,More Than This,I could feel at the time. There was no way of ...,110.0,0.3,i could feel time there way knowing fallen lea...,10000 maniacs
1,10000 Maniacs,Because The Night,"Take me now, baby, here as I am. Hold me close...",110.0,0.3,take baby i hold close try understand desire h...,10000 maniacs
2,10000 Maniacs,These Are Days,These are. These are days you'll remember. Nev...,110.0,0.3,these these day 'll remember never never since...,10000 maniacs
3,10000 Maniacs,A Campfire Song,"A lie to say, ""O my mountain has coal veins an...",110.0,0.3,a lie say o mountain coal vein bed dig .. 500 ...,10000 maniacs
4,10000 Maniacs,Everyday Is Like Sunday,Trudging slowly over wet sand. Back to the ben...,110.0,0.3,trudging slowly wet sand back bench clothes st...,10000 maniacs
...,...,...,...,...,...,...,...
87156,Sambô,Smells Like Teen Spirit,"(Chorus). Hello, hello,hello,how low. Hello,he...",71.0,0.8,( chorus ) hello hello hello low hello hello h...,sambô
87157,Sambô,Valerie,Well sometimes I go out by myself. And I look ...,71.0,0.8,well sometimes i go and i look across water an...,sambô
87158,Sambô,Wake Me Up,Feeling my way through the darkness. Guided by...,71.0,0.8,feeling way darkness guided beating heart i ca...,sambô
87159,Seu Jorge,Don't,"Don't, don't, that's what you say. Each time t...",106.0,4.2,do n't n't 's say each time i hold way when i ...,seu jorge


In [8]:
data.to_csv('data.csv', index=False)

In [ ]:
#Сейчас заранее сделаем датасет, по которому будет в самом конце определять релевантность оставшихся документов. В него войдут:
#векторное представление песни, кол-во песен у исполнителя и его популярность

In [20]:
import io
from tqdm import tqdm
from itertools import islice
import numpy as np

In [24]:
#Подготавливаем fasttext вектора 


def load_vectors(fname, limit):
  fin = io.open(fname, 'r', encoding = 'utf-8', newline = '\n', errors = 'ignore')
  n, d = map(int, fin.readline().split())
  data = {}
  for line in tqdm(islice(fin, limit), total = limit):
    tokens = line.rstrip().split(' ')
    data[tokens[0]] = np.array(list(map(float, tokens[1:])))
  return data

vecs = load_vectors('crawl-300d-2M.vec', 2000000)   

100%|█████████▉| 1999995/2000000 [03:15<00:00, 10237.39it/s]


In [480]:
with open('vecs.pkl', 'wb') as file:
    p.dump(vecs, file)

In [12]:
with open('vecs.pkl', 'rb') as file:
    vecs = p.load(file)

In [13]:
#Создаем нейтральный вектор песен как среднее всех слов 
zero_lyric = sum(vecs.values()) / len(vecs)

In [21]:
#создаем векторные представления текстов песен
dim = 300
tfidf_lyric = TfidfVectorizer()
data_lyric_vectors = tfidf_lyric.fit_transform(data['Lyric_clear'])
vocab_lyric = np.zeros((len(tfidf_lyric.vocabulary_.keys()), dim))
for key in tqdm(tfidf_lyric.vocabulary_.keys()):
    vocab_lyric[tfidf_lyric.vocabulary_[key]] = vecs.get(key, zero_lyric)
data_x = pd.DataFrame(data_lyric_vectors.dot(vocab_lyric))
data_x['Song'] = data['Song']
data_x['Popularity'] = data['Popularity']

100%|██████████| 116999/116999 [00:00<00:00, 326694.02it/s]


In [125]:
#Для имен исполнителей сделаем отдельный словарь 

from gensim.models import Word2Vec
rev = np.array(list(set(data['Artist_clear'].values)))
sent = list(map(lambda t: t.split(), rev))

model = Word2Vec(sent, min_count = 1, seed = 1)
wv = model.wv

In [482]:
#Сохраним его
with open('wv.pkl', 'wb') as file:
    p.dump(wv, file)

In [16]:
with open('wv.pkl', 'rb') as file:
    wv = p.load(file)

In [126]:
dim = 100
tfidf_art = TfidfVectorizer()
data_art_vectors = tfidf_art.fit_transform(data['Artist_clear'])
vocab_art = np.zeros((len(tfidf_art.vocabulary_.keys()), dim))
for key in tqdm(tfidf_art.vocabulary_.keys()):
    try:
        vocab_art[tfidf_art.vocabulary_[key]] = wv[key]
    except:
        vocab_art[tfidf_art.vocabulary_[key]] = np.zeros(dim)
data_x = pd.DataFrame(np.hstack((data_x,data_art_vectors.dot(vocab_art))))


100%|██████████| 1703/1703 [00:00<00:00, 340543.49it/s]


In [24]:
#Исходя из простоты нашей будущей модели и задачи в целом, я решил не использовать название песни как какой либо опознавательный 
#признак, поскольку обычно в нем содежаться stopwords или слова, которые скорее всего уже есть в нашем словаре слов. Если
#мы просто добавим обработанное название в Lyric_clear, то модель не придаст этим словам особое значение и будет относится к ним
#так же как к обычным словам песни. Другие реализации, которые приходили мне на ум, были более затратные и сложные.
#Еще
data_x

,0,1,2,3,4,5,6,7,8,9,...,392,393,394,395,396,397,398,399,400,401
0,-0.033031,-0.512715,-0.153370,-0.097609,-0.214198,0.047821,-0.303498,0.718415,0.342354,-0.163267,...,-0.008563,-0.008216,-0.011546,0.002525,-0.004374,-0.003494,0.011418,-0.001250,0.007046,-0.005402
1,-0.145488,-0.420708,-0.583316,0.096608,-0.299939,-0.253328,-0.542446,0.446627,0.167136,-0.189905,...,-0.008563,-0.008216,-0.011546,0.002525,-0.004374,-0.003494,0.011418,-0.001250,0.007046,-0.005402
2,-0.156373,-0.360358,-0.206771,0.020815,-0.459582,0.001751,-0.658485,0.052937,0.531022,-0.448962,...,-0.008563,-0.008216,-0.011546,0.002525,-0.004374,-0.003494,0.011418,-0.001250,0.007046,-0.005402
3,0.524453,-0.497657,-0.741633,-0.630234,-0.617959,-0.461555,-0.223090,-0.049476,0.931959,-0.279106,...,-0.008563,-0.008216,-0.011546,0.002525,-0.004374,-0.003494,0.011418,-0.001250,0.007046,-0.005402
4,0.299058,-0.932219,0.026010,-0.490329,-0.619244,0.372165,-0.436893,0.741499,0.679547,-0.185792,...,-0.008563,-0.008216,-0.011546,0.002525,-0.004374,-0.003494,0.011418,-0.001250,0.007046,-0.005402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87156,-0.081184,-0.392561,0.089983,-0.582103,0.078026,0.032064,-0.358876,0.016584,-0.119325,-0.244360,...,0.001686,0.002842,-0.003575,-0.005617,-0.008486,0.003800,-0.001102,0.004943,-0.006003,-0.002403
87157,-0.302187,-0.196680,0.162238,-0.185993,0.239343,0.157539,0.025123,-0.387363,0.359581,-0.278340,...,0.001686,0.002842,-0.003575,-0.005617,-0.008486,0.003800,-0.001102,0.004943,-0.006003,-0.002403
87158,0.271109,-0.708261,-0.103755,-0.062080,-0.440099,-0.063939,-0.483305,0.164237,0.842728,-0.050339,...,0.001686,0.002842,-0.003575,-0.005617,-0.008486,0.003800,-0.001102,0.004943,-0.006003,-0.002403
87159,-0.017477,-0.439919,-0.236045,-0.186003,-0.377616,0.026852,-0.301296,0.475263,0.739806,0.196694,...,0.001764,0.003781,-0.000391,0.007695,0.006636,0.000869,-0.006349,0.004903,-0.008520,-0.009375


In [478]:
#Сохраним его
with open('data_x.pkl', 'wb') as file:
    p.dump(data_x, file)

In [23]:
with open('data_x.pkl', 'rb') as file:
    data_x = p.load(file)

In [ ]:
#Далее я захотел сделать обратный индекс, но столкнулся с проблемой, что моим способом, даже с pool.map он будет выполняться
#довольно долго:

In [25]:
#кол-во слов в словаре песен
len(tfidf_lyric.vocabulary_.keys())

116999

In [ ]:
#для первых 1000 слов документы определятся за 130 секунда, значит 120000 / 1000 * 130 / 60 / 60 = 4.3 часа
#Я попробую поставить компьютер на ночь, но на вский случай подготовим план побега
#Сделаем допущение, что запрос человека состоит в среднем из 5-8 слов, ну максимум 10-15, а так же у нас интернет 3G.
#Для этих слов мы за 15 секунд найдем нужные документы и продолжим дальyейшие вычисления!
#Так же мы проверим, есть ли в запросе имя исполнителя

In [ ]:
#Но для начала сделаем обработку запросов

In [26]:
#Подготовим нужные функции для лемматизатора
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.ADJ,
        'V': wordnet.VERB,
        'N': wordnet.NOUN,
        'R': wordnet.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.NOUN

from nltk import WordNetLemmatizer
def my_lemmatizer(sent):
    lemmatizer = WordNetLemmatizer()
    tokenized_sent = sent.split()
    pos_tagged = [(word, get_wordnet_pos(tag))
                 for word, tag in pos_tag(tokenized_sent)]
    return ' '.join([lemmatizer.lemmatize(word, tag)
                    for word, tag in pos_tagged])


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
#Это для уменьшения словаря слов
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
sw0 = np.array(['.',',','!','?','`','\'','[',']','``','@','#','№','$','^','*','-','&','\\',';',':','>','<'])
sw1 = np.array(list(set(stopwords.words('english'))))
sw2 = np.append(sw1, sw0)
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
#Это будет нашим примером
query = "Air breathe how Chris "

In [29]:
query_tok = [i.lower() for i in word_tokenize(my_lemmatizer(query)) if i not in sw2]
query_tok

['air', 'breathe', 'chris']

In [30]:
def find_word_docs(word):
    global data
    docs = np.array([], dtype=np.int)
    for i in data.index:
        try:
            if((word in data['Lyric_clear'][i]) or (word in data['Artist_clear'][i])):
                docs = np.append(docs, i)
        except: 
            pass
    return docs


In [31]:
#Это подходящие документы по каждому слову
query_docs = np.array(list(map(find_word_docs, query_tok)))

In [32]:
query_docs

array([array([    5,     6,    16, ..., 87154, 87157, 87160]),
       array([    1,    13,    14, ..., 87038, 87054, 87056]),
       array([   24,    66,    73, ..., 87074, 87102, 87133])],
      dtype=object)

In [33]:
#Получаем документы, в которых есть запрошенные слова после пересечения списков
docs_itog = np.array(list(set(query_docs[0]).intersection(*query_docs[1:])))
len(docs_itog)

28

In [ ]:
#перейдем к ранжированию документов. Сначала сделаем вектор запроса, чтобы впоследствии посчитать косинусное расстояние 
#между текстом песни и запросом

In [34]:
dim = 300
query_lyric_vec = tfidf_lyric.fit_transform([' '.join(query_tok)])
vocab_lyric = np.zeros((len(tfidf_lyric.vocabulary_.keys()), dim))
for key in tqdm(tfidf_lyric.vocabulary_.keys()):
    vocab_lyric[tfidf_lyric.vocabulary_[key]] = vecs.get(key, zero_lyric)
query_lyric_x = pd.Series((query_lyric_vec.dot(vocab_lyric))[-1])

100%|██████████| 3/3 [00:00<?, ?it/s]


In [35]:
dim = 100
query_artist_vec = tfidf_lyric.fit_transform([' '.join(query_tok)])
vocab_artist = np.zeros((len(tfidf_lyric.vocabulary_.keys()), dim))
for key in tqdm(tfidf_lyric.vocabulary_.keys()):
    try:
        vocab_artist[tfidf_lyric.vocabulary_[key]] = wv[key]
    except:
        vocab_artist[tfidf_lyric.vocabulary_[key]] = np.zeros(dim)
query_artist_x = pd.Series((query_artist_vec.dot(vocab_artist))[-1])

100%|██████████| 3/3 [00:00<?, ?it/s]


In [44]:
#Формируем выборку наших отобранных документов, приведем все переменные к отрезку от 0 до 2, при этом чем больше, тем лучше
x = pd.DataFrame(index=docs_itog)
x['Song'] = data_x[300].iloc[docs_itog] 
x['Song'] = x['Song'] / x['Song'].max() * 2
x['Popularity'] = data_x[301].iloc[docs_itog]
x['Popularity'] = x['Popularity'] / x['Popularity'].max() * 2
x['cosdist_lyric'] = (cosine_distances(data_x.iloc[docs_itog,:300],[query_lyric_x.values]) - 2) * -1
x['cosdist_artist'] = (cosine_distances(data_x.iloc[docs_itog,302:],[query_artist_x.values]) - 2) * -1

In [45]:
#Здесь я применю свои превосходные знания дедукции и индукции и создам формулу, по которой росчитаем релевантность
k = np.array([1.5,3,5,8]) #Коэффиценты
x['rel'] = x.dot(k) #Итоговый скор
x['rel'] = x['rel'] / x['rel'].max()
x = x.sort_values(by=['rel'], ascending=False) #Сортируем по убыванию

In [46]:
x
#Итоговая табличка и скор справа

,Song,Popularity,cosdist_lyric,cosdist_artist,rel
52187,2.000000,0.357947,1.832752,1.729454,1.000000
52265,2.000000,0.357947,1.634773,1.729454,0.963436
52264,2.000000,0.357947,1.616966,1.729454,0.960148
52296,2.000000,0.357947,1.596529,1.729454,0.956373
52451,2.000000,0.357947,1.562668,1.729454,0.950120
52223,2.000000,0.357947,1.553027,1.729454,0.948339
52226,2.000000,0.357947,1.538662,1.729454,0.945686
53951,1.584369,2.000000,1.583270,1.065834,0.916757
54156,1.584369,2.000000,1.576366,1.065834,0.915482
54155,1.584369,2.000000,1.573705,1.065834,0.914991


In [496]:
#Это итоговый список отранжированных документов. По запросу "Air breathe how Chris " была выдана нужная песня
data.iloc[x.index]

,Artist,SName,Lyric,Song,Popularity,Lyric_clear,Artist_clear
52187,Chris Brown,No Air,Tell me how I'm supposed to breathe with no ai...,563.0,14.3,tell i 'm suppose breathe air if i die i wake ...,chris brown
52265,Chris Brown,Better On The Side,[Diddy talking]. I remember the first time i s...,563.0,14.3,diddy talking i remember first time see moonwa...,chris brown
52264,Chris Brown,Better On The Other Side,"I remember the first time I seen you moonwalk,...",563.0,14.3,i remember first time i see moonwalk i believe...,chris brown
52296,Chris Brown,Can't Say No,I can't tell you no. Cause you're irresistible...,563.0,14.3,i ca n't tell cause 're irresistible ( no ) ve...,chris brown
52451,Chris Brown,I'll Be Right Here,"[Verse 1]. I hope you notice how I feel,. Girl...",563.0,14.3,verse 1 i hope notice i feel girl i ai n't run...,chris brown
52223,Chris Brown,Actin' Up,Back up. Shout to Justin Bieber. Chris Brown. ...,563.0,14.3,back shout justin bieber chris brown base loud...,chris brown
52226,Chris Brown,Ain't Said Nothing,See I came in it to see her. And I ain't tryna...,563.0,14.3,see i come see and i ai n't tryna leave alone ...,chris brown
53951,Eminem,"Bad Meets Evil (feat. Royce da 5'9"")",[intro]. [Cowboy]. I reckon you ain't familiar...,446.0,79.9,intro cowboy i reckon ai n't familiar parts yo...,eminem
54156,Eminem,Off The Wall (Feat. Redman),Yo. (Yo). Look!. (Yo). (Chorus 1). Eminem-. No...,446.0,79.9,yo ( yo ) look ( yo ) ( chorus 1 ) eminem- no ...,eminem
54155,Eminem,Off the Wall,[Eminem]. No matter what people say. I'm gon k...,446.0,79.9,eminem no matter people say i 'm gon keep rapp...,eminem


In [41]:
#А теперь я раздам релевантность каждому из этих документов вручную и мы обучим модель, чтобы получить более менее невыдуманные 
#коэффиценты
y = pd.DataFrame(x['rel'])
y['index'] = np.arange(28)
y['docs_rel'] = np.array([10,9,9,8,7,8,8,6,8,7,8,6,6,10,7,5,7,3,5,3,5,7,4,6,3,1,1,2])
x = x.drop('rel', axis=1)

In [ ]:
#Будем обучать линейную регрессию и обучать на метрике nDCG@K, поскольку в ней можно использовать релевантность документов не
#только 0 или 1. Так же она приведена к интервалу от 0 до 1.

In [395]:
def nDCGaK(y_pred, y_test):
    itog = 0
    itog_max = 0
    for i in range(len(y_pred)):
        itog += (2**(y_pred.iloc[i]) - 1)/(np.log(i+2) / np.log(2))
    for i in range(len(y_test)):
        itog_max += (2**(y_test.iloc[i]) - 1)/(np.log(i+2) / np.log(2))
    return itog / itog_max

In [387]:
#Обучим самую простую модельку
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [474]:
kf = KFold(n_splits=5)
metric = np.array([])
first = True

for tr_ind, tes_ind in kf.split(x):
    mod = LinearRegression(positive=True)
    mod.fit(x.iloc[tr_ind], y['docs_rel'].iloc[tr_ind])
    y_pred = pd.Series(mod.predict(x.iloc[tes_ind]), index=y['docs_rel'].iloc[tes_ind].index)
    y_pred = y_pred.sort_values(ascending=False)
    metric = np.append(metric, nDCGaK(y.loc[y_pred.index]['docs_rel'], y.loc[y_pred.index]['docs_rel'].sort_values(ascending=False)))
    if(first == True):
        first = False
        koefs = np.array(mod.coef_)
    else:
        koefs = np.vstack((koefs,mod.coef_))

In [475]:
#Итоговая метрика
metric.mean()

0.8922037970254448

In [476]:
#Модель посчитала, что схожесть запроса по названию и кол-во песен у исполнителя имеет примерно одинаковый вес. Интереесно!
k_itog = koefs.mean(axis=0)
k_itog

array([ 1.55735732,  0.33114107, 13.83990411,  1.6114938 ])

In [ ]:
#Теперь у нас есть все, чтобы запустить наш поисковик на веб страничке!

In [ ]:
#далее немножко мусорного кода про прямой индекс, который я перенес в server.py, чтобы он ошибки не выдовал
#этот индекс я так же сделал заранее

In [66]:
class Document:
    def __init__(self, title, text):
        self.title = title
        self.text = text
    
    def format(self):
        return [self.title, self.text + ' ...']

In [67]:
index = []


for i in data.index:
    index.append(Document(f"{data['Artist'][i]} — {data['SName'][i]}", f"{data['Lyric'][i][:80]}"))


In [55]:
with open('index.pkl', 'wb') as file:
    p.dump(index, file)

In [56]:
with open('index.pkl', 'rb') as file:
    index = p.load(file)

In [68]:
results = [index[doc].format() for doc in docs_itog]

In [ ]:
def build_index():
    global index, data
    for i in data.index:
        straight_index.append(Document(f"{data['Artist'][i]} — {data['SName'][i]}", f"{data['Lyric'][i][:80]}"))
